# Stokes equations with stabilized first order elements

## Implementation

First, the :pydolfin module is imported:

In [ ]:
from dolfin import *

In this example, different boundary conditions are prescribed on
different parts of the boundaries. This information must be made
available to the solver. One way of doing this, is to tag the different
sub-regions with different (integer) labels. DOLFIN provides a class
:pyMeshFunction &lt;dolfin.cpp.mesh.MeshFunction&gt; which is useful for
these types of operations: instances of this class represent functions
over mesh entities (such as over cells or over facets). Mesh and mesh
functions can be read from file in the following way:

In [ ]:
# Load mesh and subdomains
mesh = Mesh("dolfin_fine.xml.gz")
sub_domains = MeshFunction("size_t", mesh, "dolfin_fine_subdomains.xml.gz")

Next, we define a :pyMixedFunctionSpace
&lt;dolfin.functions.functionspace.MixedFunctionSpace&gt; composed of a
:pyVectorFunctionSpace
&lt;dolfin.functions.functionspace.VectorFunctionSpace&gt; and a
:pyFunctionSpace &lt;dolfin.cpp.function.FunctionSpace&gt;, both of
continuous piecewise linears.

In [ ]:
# Define function spaces
scalar = FunctionSpace(mesh, "CG", 1)
vector = VectorFunctionSpace(mesh, "CG", 1)
system = vector * scalar

Now that we have our mixed function space and marked subdomains defining
the boundaries, we create functions for the boundary conditions and
define boundary conditions:

In [ ]:
# Create functions for boundary conditions
noslip = Constant((0, 0))
inflow = Expression(("-sin(x[1]*pi)", "0"))
zero   = Constant(0)

# No-slip boundary condition for velocity
bc0 = DirichletBC(system.sub(0), noslip, sub_domains, 0)

# Inflow boundary condition for velocity
bc1 = DirichletBC(system.sub(0), inflow, sub_domains, 1)

# Collect boundary conditions
bcs = [bc0, bc1]

Here, we have given four arguments to :pyDirichletBC
&lt;dolfin.cpp.fem.DirichletBC&gt;. The first specifies the
:pyFunctionSpace &lt;dolfin.cpp.function.FunctionSpace&gt;. Since we
have a :pyMixedFunctionSpace
&lt;dolfin.functions.functionspace.MixedFunctionSpace&gt;, we write
system.sub(0) for the :pyVectorFunctionSpace
&lt;dolfin.functions.functionspace.VectorFunctionSpace&gt;, and
system.sub(1) for the :pyFunctionSpace
&lt;dolfin.cpp.function.FunctionSpace&gt;. The second argument specifies
the value on the Dirichlet boundary. The two last arguments specify the
marking of the subdomains; sub\_domains contains the subdomain markers
and the number given as the last argument is the subdomain index.

The bilinear and linear forms corresponding to the stabilized weak mixed
formulation of the Stokes equations are defined as follows:

In [ ]:
# Define variational problem
(v, q) = TestFunctions(system)
(u, p) = TrialFunctions(system)
f = Constant((0, 0))
h = CellSize(mesh)
beta  = 0.2
delta = beta*h*h
a = (inner(grad(v), grad(u)) - div(v)*p + q*div(u) + \
    delta*inner(grad(q), grad(p)))*dx
L = inner(v + delta*grad(q), f)*dx

To compute the solution we use the bilinear and linear forms, and the
boundary condition, but we also need to create a :pyFunction
&lt;dolfin.cpp.function.Function&gt; to store the solution(s). The
(full) solution will be stored in `w`, which we initialize using the
MixedFunctionSpace system. The actual computation is performed by
calling solve with the arguments `a`, `L` and `bcs`. The separate
components `u` and `p` of the solution can be extracted by calling the
:pysplit
&lt;dolfin.functions.function.Function.split&gt; function.

In [ ]:
# Compute solution
w = Function(system)
solve(a == L, w, bcs)
u, p = w.split()

Finally, we can store to file and plot the solutions.

In [ ]:
# Save solution in VTK format
ufile_pvd = File("velocity.pvd")
ufile_pvd << u
pfile_pvd = File("pressure.pvd")
pfile_pvd << p

# Plot solution
plot(u)
plot(p)
interactive()

## Complete code